In [210]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144
import pandas as pd
import numpy as np
import re
import traceback
from multiprocessing import Pool
pool = Pool(processes=16)
import requests
import dill
from bs4 import BeautifulSoup
from datetime import datetime
import operator
from operator import itemgetter

In [90]:
import itertools
import networkx as nx

In [22]:
from static_grader import grader

In [2]:
import pickle as pk
def dump_file(filename,data):
    with open(filename,'wb') as file:
        pk.dump(data,file)
def load_file(filename):
    with open(filename,'rb') as file:
        return pk.load(file)
from static_grader import grader    

In [3]:
link_list=load_file('link_list')

In [5]:
def myhtmlcall(url):
    
    return requests.get(url).text
    
p = Pool(24)
newobj  = p.map(myhtmlcall, link_list[0])

In [6]:
newobj[0][:100]

'<!DOCTYPE html>\n  <!--[if IEMobile 7]><html class="no-js ie iem7" lang="en" dir="ltr"><![endif]-->\n '

In [7]:
html_dict={}
for i,j in zip(link_list[0],newobj):
    html_dict[i]=j

In [10]:
dump_file('html_dict',html_dict)

In [12]:
html_dict=load_file('html_dict')

In [20]:
def get_captions(page):
    soup=BeautifulSoup(page,'lxml')
    caption_divs= soup.find_all('font', attrs={'face': 'Verdana, Arial, Helvetica, sans-serif'}) 
    caption_divs.extend(soup.findAll('div', attrs={'class':'photocaption'}))
    caption_divs.extend(soup.find_all("td", {'valign': 'top', 'class': 'photocaption'}, text=True))
    caption_divs.extend(soup.find_all("td", {'valign': 'top', 'class': 'photocaption', 'style': 'background-color:#faf9ee'}, text=True))
      
    return [i.text for i in caption_divs]
    

In [29]:
page=requests.get('http://www.newyorksocialdiary.com/party-pictures/2015/celebrating-the-neighborhood').text
test_caption=get_captions(page)

In [30]:
print(len(test_caption))

109


In [54]:
#https://github.com/hazlanhamdan/social_graph/blob/master/graph_analyze.py

def extract_names(caption):
    '''
    Extract a list of names from a caption string.  
    Names should be in format 'Firstname Lastname'.  
    If no names extracted, return None.  
    This function splits up the string using 'when', 'and', and commas
    and passes the resulting segments to the function `parse_names`
    '''
    # Remove leading and trailing whitespace
    match = re.search('^\s*([^\s].+[^\s])\s*$', caption, re.DOTALL)
    if match:
        caption = match.group(1)
    else: 
        print (caption)
        print ('NO CHARACTERS FOUND')
        return None
    
    # If too long, throw out the caption
    if len(caption) > 250:
        print ('TOO LONG')
        return None
        
    # initialize the list
    names = []
    
    # split on pattern '... with ...'
    # if there's a match, parse the segment before the 'with',
    # leave the rest to be split below
    segments = re.split('\swith\s', caption)
    if len(segments)==2:
        new_names = parse_names(segments.pop(0))
        if new_names:
            for nm in new_names:
                if nm != None: names.append(nm)
    elif len(segments)>2:
        print ('***** UNHANDLED CASE: MORE THAN 1 "WITH" *****')
        print (caption)
        return None
    # after this, segments have length 1
        
    # split on the pattern 'name, name, and name'
    segments = re.split('\s?,\s?(?:and)?\s?', segments[0])
        
    # if not split already, split on pattern 'name and name',
    # but ONLY IF it's not a 'jack and jane doe' case
    if len(segments)==1:
        test_segs = re.split('\sand\s', segments[0])
        # 'jack and jane doe' case indicated by no whitespace in 
        # the first test_seg
        if re.search('\s', test_segs[0]):
            segments = test_segs
    
    #parse all the segments
    for sg in segments:
        new_names = parse_names(sg)
        if new_names:
            for nm in new_names:
                if nm != None: names.append(nm)
    
    return names

def parse_names(seg):
    '''
    Parse the names in segments generated in `extract_names`.
    If parsing fails, return None.
    Except for a few special cases, each segment is assumed to contain
    only one name. 
    '''
    # remove text in parentheses
    seg = re.sub(r'^\([^\)]*\)\s', '', seg) # bol
    seg = re.sub(r'\s\([^\)]*\)\.?$', '', seg) # eol
    seg = re.sub(r'\s\([^\)]*\)\s', ' ', seg) # elsewhere
    
    # All subsequent processing is word by word....
    words = seg.split()
    
    titles = ('Dr.', 'Mayor', 'Mr.', 'Mrs.', 'Ms.', 'Sir', 'Justice', 'Governor'
              'Prince', 'Princess', 'King', 'Queen', 'Count', 'Countess', 'Baroness')
    
    flags = ('Honoree','Chair','Co-Chair','Trustee','Director','Chairman','Designer','Administrator','Winner','Fellow',
             'Actress','Playwright','Curator','Laureate','Member','Governor','Representative','Host','Vice-Chair',
             'Rev.','Reverend','Conductor','Mayor','Co-Chairs','Lady','Designer/artist','President','Recipient','Editor',
             'Commissioner','General','Speaker','Teacher','Sgt.')
    if len(words)>2 and (words[-3] in flags or re.search(r':$', words[-3]) or re.search(r"'s", words[-3])):
        words = words[-2:]
    elif len(words)>3 and (words[-4] in flags or re.search(r':$', words[-4]) or re.search(r"'s", words[-4])):
        words = words[-3:]
    
    # Special case: '[Tt]he Honorable'
    if len(words)==4 and (words[0]=='the' or words[0]=='The') and words[1]=='Honorable':
        return [' '.join([words[2], words[3]])]
    
    # Special case: 'jack and jane doe'
    if len(words)==4 and words[1]=='and':
        if not all(check_caps(wd) for wd in [words[0],words[2],words[3]]):
            return None
        return [' '.join([words[0], words[3]]), ' '.join([words[2], words[3]])]
    
    # Special case: 'mr. and mrs. jane doe'
    if len(words)==5 and words[0] in titles and words[2] in titles and words[1]=='and':
        if not all(check_caps(wd) for wd in words[3:]):
            return None
        return [' '.join([words[3], words[4]])]
    
    # If not capitalized like a name, throw out
    if not all(check_caps(wd) for wd in words):
        return [None]
    
    if len(words)==1:
        return [None]
    elif len(words)==2:
        return [' '.join(words)]
    elif len(words)==3:
        if words[0] in titles:
            return [' '.join(words[1:])]
        else:
            return [' '.join([words[0], words[2]])]
    elif len(words)==4 and words[0] in titles:
        return [' '.join([words[1], words[3]])]
    else:
        print ()
        print ('***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****')
        print ()
        #print seg
        return [None]

def check_caps(word):
    '''
    Return a boolean indicating whether a putative word in a name follows 
    the right capitalization conventions
    We require that the first alphanumeric character be uppercase (leading
    quotations and other punctuation are allowed) and that it is followed 
    by at least one lowercase character.  
    '''
    # get the first letter, False if none found
    match = re.search('\w', word)
    if not match: 
        return False
    first_let = match.group()
    # First letter must be uppercase
    if not first_let.isupper():
        return False
    # There must be at least one lowercase letter
    match = re.search('[a-z]', word)
    if not match: 
        return False
    return True

In [39]:
#test_names=p.map(get_captions,test_case[:2])

In [59]:
Name1=list(map(extract_names,test_case))


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****



In [65]:
Name1[:10]

[['Glenn Adamson',
  'Simon Doonan',
  'Craig Leavitt',
  'Jerome Chazen',
  'Andi Potamkin',
  'Ralph Pucci',
  'Kirsten Bailey',
  'Edwin Hathaway'],
 ['Randy Takian'],
 ['Kamie Lightburn', 'Christopher Spitzmiller'],
 ['Christopher Spitzmiller', 'Diana Quasha'],
 ['Mariam Azarm', 'Sana Sabbagh', 'Lynette Dallas'],
 ['Christopher Spitzmiller', 'Sydney Shuman', 'Matthew Bees'],
 ['Christopher Spitzmiller', 'Tom Edelman'],
 ['Warren Scharf', 'Sydney Shuman'],
 ['Amory McAndrew', 'Sean McAndrew'],
 ['Sydney Shuman', 'Mario Buatta', 'Helene Tilney']]

In [61]:
Name2=list(set(sum(Name1,[])))

In [62]:
Name2.sort()
Name2[:10]

['Alberto Villalobos',
 'Alex Papachristidis',
 'Alexandra Richards',
 'Alexis Mersentes',
 'Allen Swerdlick',
 'Allison Minton',
 'Amory McAndrew',
 'Andi Potamkin',
 'Angela Clofine',
 'Anne Strickland']

In [64]:
def sample_names():
    return Name2[:100]

grader.score(question_name='graph__sample_names', func=sample_names)

Your score:  0.9500000000000006


In [72]:
sub_captions=list(map(get_captions,list(html_dict.values())[:2]))

In [73]:
sub_captions[0]

['Tatiana Boncampagni',
 'Annalise Peterson',
 'Jessie Maxwell and Jason Eblances',
 'Amy McFarland and Arden Wohl',
 'Sylvana Soto-Ward and Luisiana Mendoza',
 'Frederic Fekkai and Shirin von Wulffen',
 'Jennifer Creel, Mark Gilbertson, and Ashley McDermott',
 'Leven Rambin, Darcy Jones, and Tracy Stern',
 'Olivia Chantecaille and Adelina Wong Ettelson',
 'Olivia Palermo and Susan Shin',
 'Kenya Clarke, Monique Jones, and Mary Brown',
 'Peggy Siegal',
 'Gillian Miniter and Arthur Bakal',
 'Kim Hicks and Cherell Flaherty',
 'Cellphone cuisine',
 'Sara Ayres ',
 'Vogue cover model Doutzen Kroes',
 'L. to r.: Amy Smilovic, Emilia Fanjul Pfeifler, and Liz Walker; Zani Gugelmann and Rachel Roy. ',
 'Andrew Saffir and Nina Griscom ',
 'Damon Dash',
 'Elizabeth Meigher and Bonnie Morrison ',
 'Byrdie Bell, Edward Barsamian, and Serena Nakkahah',
 'Dayssi Olarte de Kanavos and Ashley McDermott ',
 'Chessy Wilson and Lily Maddock',
 'Andrew and Rachel',
 'Billy Norwich and  Andre Balazs ',
 'S

In [74]:
def get_all_captions(html_dict):
    return list(map(get_captions,list(html_dict.values())))

In [77]:
All_captions=get_all_captions(html_dict)

In [87]:
All_captions[0][:10]

['Tatiana Boncampagni',
 'Annalise Peterson',
 'Jessie Maxwell and Jason Eblances',
 'Amy McFarland and Arden Wohl',
 'Sylvana Soto-Ward and Luisiana Mendoza',
 'Frederic Fekkai and Shirin von Wulffen',
 'Jennifer Creel, Mark Gilbertson, and Ashley McDermott',
 'Leven Rambin, Darcy Jones, and Tracy Stern',
 'Olivia Chantecaille and Adelina Wong Ettelson',
 'Olivia Palermo and Susan Shin']

In [79]:
dump_file('All_captions',All_captions)

In [83]:
sub_caption_test=list(map(extract_names,All_captions[0]))


[['Lauren Ezersky', 'Jane Hobson'],
 ['Michele Klein'],
 ['Bill Berloni'],
 ['Roberto Dutesco'],
 ['Sylvia Plachy'],
 [],
 ['Cornelia Guest', 'Aldon Jr.'],
 ['Harriette Katz', 'Jeffrey Selden'],
 [],
 []]

In [84]:
def extract_all_names(All_captions):
    All_names=[]
    for acaption in All_captions:
        All_names.append(list(map(extract_names,acaption)))
    return All_names
        

In [85]:
All_names=extract_all_names(All_captions)

***** UNHANDLED CASE: MORE THAN 1 "WITH" *****
Lara getting ready in the bridal suite with a little help from with her mother and sister

***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: 


NO CHARACTERS FOUND
 
NO CHARACTERS FOUND

NO CHARACTERS FOUND

***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


NO CHARACTERS FOUND

***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


NO CHARACTERS FOUND

***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****

TOO LONG
TOO LONG

***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


NO CHARACTERS FOUND

NO CHARACTERS FOUND

***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


NO CHARACTERS FOUND

***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: 


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****

 
NO CHARACTERS FOUND

***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****

 
NO CHARACTERS FOUND
 
NO CHARACTERS FOUND
 
NO CHARACTERS FOUND
 
NO CHARACTERS FOUND

NO CHARACTERS FOUND
 
NO CHARACTERS FOUND
 
NO CHARACTERS FOUND
 
NO CHARACTERS FOUND
 
NO CHARACTERS FOUND
 
NO CHARACTERS FOUND

***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****

 
NO CHARACTERS FOUND
 
NO CHARACTERS FOUND
 
NO CHARACTERS FOUND
 
NO CHARACTERS FOUND
 
NO CHARACTERS FOUND

***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****

TOO LONG
 
NO CHARACTERS FOUND
 



***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****

TOO LONG

***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****

 
NO CHARACTERS FOUND

***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****

 
NO CHARACTERS FOUND
 
NO CHARACTERS FOUND
 
NO CHARACTERS FOUND
TOO LON



NO CHARACTERS FOUND

***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


NO CHARACTERS FOUND

NO CHARACTERS FOUND

***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


NO CHARACTERS FOUND

NO CHARACTERS FOUND

***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


NO CHARACTERS FOUND

NO CHARACTERS FOUND

***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


NO CHARACTERS FOUND

NO CHARACTERS FOUND

***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


NO CHARACTERS FOUND

***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


NO CHARACTERS FOUND

NO CHARACTERS FOUND

NO CHARACTERS FOUND

***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


NO CHARACTERS FOUND

NO CHARACTERS FOUND

***** UNHAN


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****

 
NO CHARACTERS FOUND

***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****

 C3 
NO CHARACTERS FOUND

***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****





***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****

TOO LONG

***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****

TOO LONG

***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****

***** UNHANDLED CASE: MORE THAN 1 "WITH" *****
Mark and Kim Standish (pictured here with JD Hoye, President of the National Academy Foundation), honored along with RBC Capital Markets
TOO LONG

***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****

 
NO CHARACTERS FOUND
 
NO CHARACTERS FOUND
 
NO CHARACTERS FOUND

***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED **


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****

TOO LONG
TOO LONG
TOO LONG

***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


NO CHARACTERS FOUND

NO CHARACTERS FOUND

NO CHARACTERS FOUND

NO CHARACTERS FOUND

NO CHARACTERS FOUND

NO CHARACTERS FOUND

NO CHARACTERS FOUND

***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


NO CHARACTERS FOUND

***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****

TOO LONG

***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****

TOO LONG

***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****

 
NO CHARACTERS FOUND

***** UNHANDLED CASE: SEGMENT PATTER

In [86]:
print(len(All_names),len(All_captions))

1193 1193


In [88]:
#dump_file('All_names',All_names)

In [89]:
print(len(sum(All_names,[])))

102758


In [117]:
class Graph:
    def __init__(self):
        self.graph=nx.Graph()
        self.pair=[]
    
    def _gen_edges(self,alist):
        pairs=list(itertools.combinations(alist, 2))
        self.pair.append(pairs)
        
        return pairs
    
    def build_graph(self,alist):
        edges=self._gen_edges(alist)
        self.graph.add_edges_from(edges)
        
    def multi_build(self,lists):
        try:
            for alist in lists:
                for single_list in alist:
                    self.build_graph(single_list)
        except Exception as e:
            print("type error: " + str(e))
            print(traceback.format_exc())
        

In [151]:
people_names=sum(All_names,[])

In [159]:
All_names[0][:30]

[['Tatiana Boncampagni'],
 ['Annalise Peterson'],
 ['Jessie Maxwell', 'Jason Eblances'],
 ['Amy McFarland', 'Arden Wohl'],
 ['Sylvana Soto-Ward', 'Luisiana Mendoza'],
 ['Frederic Fekkai'],
 ['Jennifer Creel', 'Mark Gilbertson', 'Ashley McDermott'],
 ['Leven Rambin', 'Darcy Jones', 'Tracy Stern'],
 ['Olivia Chantecaille', 'Adelina Ettelson'],
 ['Olivia Palermo', 'Susan Shin'],
 ['Kenya Clarke', 'Monique Jones', 'Mary Brown'],
 ['Peggy Siegal'],
 ['Gillian Miniter', 'Arthur Bakal'],
 ['Kim Hicks', 'Cherell Flaherty'],
 [],
 ['Sara Ayres'],
 [],
 ['Amy Smilovic', 'Emilia Pfeifler'],
 ['Andrew Saffir', 'Nina Griscom'],
 ['Damon Dash'],
 ['Elizabeth Meigher', 'Bonnie Morrison'],
 ['Byrdie Bell', 'Edward Barsamian', 'Serena Nakkahah'],
 ['Ashley McDermott'],
 ['Chessy Wilson', 'Lily Maddock'],
 [],
 ['Billy Norwich', 'Andre Balazs'],
 ['Stephanie Thompson', 'Aida Khoursheed'],
 ['Gene Seidman', 'Linda Thompson', 'Tom Krizmanic'],
 ['Nathaniel Shuman', 'David Shuman', 'Ruth Shuman', 'Michael 

In [152]:
len(people_names)

102758

In [153]:
data_combo = []
for line in people_names:
 
    try:
        if str(line) == str(['']):
            pass #print len(line) #pass
        else:
            if len(line) == 1:
                pass
            else:
                #print "yes", line
                k=(list(itertools.combinations(line,2)))    #creating all possible two tuple combinations 
                                                            #of names that are in a picture 
                data_combo.append(k)
                # added a statement not to append if one name only
                # think I did this twice then! data vs list_of_words
    
    except:   
        pass

In [160]:
data_combo[:30]

[[('Jessie Maxwell', 'Jason Eblances')],
 [('Amy McFarland', 'Arden Wohl')],
 [('Sylvana Soto-Ward', 'Luisiana Mendoza')],
 [('Jennifer Creel', 'Mark Gilbertson'),
  ('Jennifer Creel', 'Ashley McDermott'),
  ('Mark Gilbertson', 'Ashley McDermott')],
 [('Leven Rambin', 'Darcy Jones'),
  ('Leven Rambin', 'Tracy Stern'),
  ('Darcy Jones', 'Tracy Stern')],
 [('Olivia Chantecaille', 'Adelina Ettelson')],
 [('Olivia Palermo', 'Susan Shin')],
 [('Kenya Clarke', 'Monique Jones'),
  ('Kenya Clarke', 'Mary Brown'),
  ('Monique Jones', 'Mary Brown')],
 [('Gillian Miniter', 'Arthur Bakal')],
 [('Kim Hicks', 'Cherell Flaherty')],
 [],
 [],
 [('Amy Smilovic', 'Emilia Pfeifler')],
 [('Andrew Saffir', 'Nina Griscom')],
 [('Elizabeth Meigher', 'Bonnie Morrison')],
 [('Byrdie Bell', 'Edward Barsamian'),
  ('Byrdie Bell', 'Serena Nakkahah'),
  ('Edward Barsamian', 'Serena Nakkahah')],
 [('Chessy Wilson', 'Lily Maddock')],
 [],
 [('Billy Norwich', 'Andre Balazs')],
 [('Stephanie Thompson', 'Aida Khourshee

In [161]:
# Create graph
import operator
eM = nx.MultiGraph()
Gee = nx.Graph()


for line in data_combo:
    for node in line:
        try:                  
                eM.add_node(node[0])
                eM.add_node(node[1])
                eM.add_edge(node[0], node[1], weight = 1)
                
        except Exception as e:
            print (str(e))
            pass

In [162]:
# calculate best friends - or the edges with highest weights
w=0
name1=[]
name2=[]
wt=[]
for u,v,data in eM.edges(data=True):
    w = data['weight']
    if Gee.has_edge(u,v):
        Gee[u][v]['weight'] += w
    else:
        Gee.add_edge(u, v, weight=w)
        
for line in Gee.edges(data=True):
    if (str(line[0])!='') and (str(line[0])!=''):        
        name1.append(str(line[0]))
        name2.append(str(line[1]))
        k=line[2].get('weight')
        wt.append(k)

In [163]:
edges_mapped=list(zip(list(zip(name1,name2)),wt))

In [164]:
edges_mapped[0:10]

[(('Jessie Maxwell', 'Jason Eblances'), 1),
 (('Amy McFarland', 'Arden Wohl'), 2),
 (('Amy McFarland', 'David Stark'), 1),
 (('Amy McFarland', 'Marisa Brown'), 3),
 (('Amy McFarland', 'Ulrica Lanaro'), 1),
 (('Amy McFarland', 'Marisa Arredondo'), 1),
 (('Amy McFarland', 'Sean Kingston'), 1),
 (('Amy McFarland', 'Susan Magazine'), 9),
 (('Amy McFarland', 'Rory Hermelee'), 1),
 (('Amy McFarland', 'Allison Aston'), 6)]

In [165]:
edges_sorted=[]
edges_sorted = sorted(edges_mapped,key=lambda x:(x[1]), reverse=True)


In [166]:
edges_sorted[:10]

[(('Gillian Miniter', 'Sylvester Miniter'), 101),
 (('Bonnie Comley', 'Stewart Lane'), 74),
 (('Jamee Gregory', 'Peter Gregory'), 71),
 (('Andrew Saffir', 'Daniel Benedict'), 66),
 (('Geoffrey Bradfield', 'Roric Tobin'), 63),
 (('Barbara Tober', 'Donald Tober'), 53),
 (('Somers Farkas', 'Jonathan Farkas'), 52),
 (('Hilary Ross', 'Wilbur Ross'), 52),
 (('Tatiana Platt', 'Campion Platt'), 47),
 (('Jean Shafiroff', 'Martin Shafiroff'), 47)]

In [226]:
Degree=list(eM.degree)
Sorted_degree=sorted(Degree,key= lambda x:(-x[1]))
Sorted_degree[:10]
                     

[('Jean Shafiroff', 621),
 ('Gillian Miniter', 495),
 ('Mark Gilbertson', 490),
 ('Geoffrey Bradfield', 350),
 ('Alexandra Lebenthal', 338),
 ('Somers Farkas', 316),
 ('Andrew Saffir', 287),
 ('Debbie Bancroft', 286),
 ('Yaz Hernandez', 282),
 ('Eleanora Kennedy', 277)]

In [182]:
dump_file('Sorted_degree',Sorted_degree)

In [227]:
def degree():
    return Sorted_degree[:100]

grader.score(question_name='graph__degree', func=degree)

Your score:  0.8900000000000006


In [203]:

pgrank=[]
pgsorted=[]
pgrank=nx.pagerank(Gee,alpha=0.55)
pgsorted = sorted(pgrank.items(), key=operator.itemgetter(1), reverse=True)
print (pgsorted[1:10])

[('Mark Gilbertson', 0.0003087931810932036), ('Gillian Miniter', 0.00025034203506211876), ('Geoffrey Bradfield', 0.00022658557967820286), ('Andrew Saffir', 0.00021754925454380898), ('Alexandra Lebenthal', 0.00019524157090549635), ('Mario Buatta', 0.00019419620181394158), ('Sharon Bush', 0.00018785837478763967), ('Michael Bloomberg', 0.00017696809301794039), ('Yaz Hernandez', 0.0001727301558884119)]


In [205]:
dump_file('pgsorted',pgsorted)

In [204]:
def pagerank():
    return pgsorted[:100]

grader.score(question_name='graph__pagerank', func=pagerank)

Your score:  0.9100000000000006


In [209]:
edges_sorted[:10]

[(('Gillian Miniter', 'Sylvester Miniter'), 101),
 (('Bonnie Comley', 'Stewart Lane'), 74),
 (('Jamee Gregory', 'Peter Gregory'), 71),
 (('Andrew Saffir', 'Daniel Benedict'), 66),
 (('Geoffrey Bradfield', 'Roric Tobin'), 63),
 (('Barbara Tober', 'Donald Tober'), 53),
 (('Somers Farkas', 'Jonathan Farkas'), 52),
 (('Hilary Ross', 'Wilbur Ross'), 52),
 (('Tatiana Platt', 'Campion Platt'), 47),
 (('Jean Shafiroff', 'Martin Shafiroff'), 47)]

In [174]:
def best_friends():
    return edges_sorted[:100]

grader.score(question_name='graph__best_friends', func=best_friends)

Your score:  0.9500000000000006


In [225]:
dump_file('edges_sorted',edges_sorted)